<a href="https://colab.research.google.com/github/Sanskar6877/BTP/blob/main/Sbic_fit_explain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers lime pandas
!pip install tensorflow_text
import tensorflow_text as text
import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from google.colab import files
import json
from lime.lime_text import LimeTextExplainer
from transformers import TFBertModel, BertTokenizer

# Choose the file from your local machine
uploaded = files.upload()

# Read the JSON file with explicit encoding specification
json_filename = list(uploaded.keys())[0]
with open(json_filename, 'r', encoding='utf-8', errors='replace') as file:
    data = json.load(file)

# Convert the loaded JSON data to a DataFrame
df = pd.DataFrame.from_dict(data, orient='index')

# Create a binary 'hate' column based on the 'label_off' column
df['hate'] = df['label_off'].apply(lambda x: 0 if x == 'not offensive' else 1)
texts = df['post'].tolist()
labels = df['label_off'].tolist()

class_names = ['not hate', 'hate']
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(df['post'], df['hate'], stratify=df['hate'])

# BERT preprocessor and encoder
bert_preprocessor = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='Inputs')
preprocessed_text = bert_preprocessor(text_input)
embeed = bert_encoder(preprocessed_text)
dropout = tf.keras.layers.Dropout(0.1, name='Dropout')(embeed['pooled_output'])
outputs = tf.keras.layers.Dense(1, activation='sigmoid', name='Dense')(dropout)

# Build and compile the model
model = tf.keras.Model(inputs=[text_input], outputs=[outputs])

Metrics = [tf.keras.metrics.BinaryAccuracy(name='accuracy'),
           tf.keras.metrics.Precision(name='precision'),
           tf.keras.metrics.Recall(name='recall')]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=Metrics)

# Train the model
history = model.fit(X_train, y_train, epochs=10)

# Evaluate the model
model.evaluate(X_test, y_test)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283835 sha256=fef171b537545b1d8fbbd8bebae34c3a72a9998c7d3e57c1563950c9ed866f28
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 15.2 MB/s eta 0:00:00


Saving sbic_valid.json to sbic_valid.json
Epoch 1/10
110/110 [==============================] - 46s 308ms/step - loss: 0.6936 - accuracy: 0.5484 - precision: 0.5442 - recall: 0.5367
Epoch 2/10
110/110 [==============================] - 36s 328ms/step - loss: 0.6567 - accuracy: 0.6107 - precision: 0.6065 - recall: 0.6072
Epoch 3/10
110/110 [==============================] - 38s 347ms/step - loss: 0.6390 - accuracy: 0.6396 - precision: 0.6407 - recall: 0.6181
Epoch 4/10
110/110 [==============================] - 37s 336ms/step - loss: 0.6249 - accuracy: 0.6590 - precision: 0.6504 - recall: 0.6719
Epoch 5/10
110/110 [==============================] - 38s 342ms/step - loss: 0.6164 - accuracy: 0.6670 - precision: 0.6647 - recall: 0.6597
Epoch 6/10
110/110 [==============================] - 37s 340ms/step - loss: 0.6086 - accuracy: 0.6725 - precision: 0.6718 - recall: 0.6609
Epoch 7/10
110/110 [==============================] - 37s 340ms/step - loss: 0.5990 - accuracy: 0.6736 - precision: 0.

TypeError: Inputs to a layer should be tensors. Got ' 
Red Nation Rising
‏

 
@RedNationRising
 
MYSTERY: Awan Brothers Court Date Disappears From Calendar After Being Postponed Twice' (of type <class 'str'>) as input for layer 'model'.

In [ ]:
#X_test[0]
def func(STR) :
    x = model.predict(np.array([STR]))
    return x


In [ ]:
explainer = LimeTextExplainer(class_names=class_names)

# Choose a specific text instance for explanation (e.g., the first one)
STR = texts[0]
exp = explainer.explain_instance(STR, func , num_features=10, num_samples=1)
%matplotlib inline
fig = exp.as_pyplot_figure()
exp.show_in_notebook(text=True)


1/1 [==============================] - 0s 145ms/step


IndexError: index 1 is out of bounds for axis 1 with size 1

In [ ]:
import numpy as np
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
def predict_probab(text):
    inputs = tokenizer.encode_plus(text, add_special_tokens=True, max_length=512, truncation=True, padding='max_length', return_token_type_ids=True, return_attention_mask=True, return_tensors='tf')
    inputs['input_ids'] = tf.cast(inputs['input_ids'], tf.int32)
    outputs = model(inputs['input_ids'])
    prob_not_hate = outputs.numpy()[0][0]
    prob_hate = 1 - prob_not_hate
    return np.array([[prob_not_hate, prob_hate]])



def predict_function(texts):
    return np.array([predict_probab(text) for text in texts])

exp = explainer.explain_instance(STR, predict_function, num_features=10, num_samples=1)

UnimplementedError: Exception encountered when calling layer 'model' (type Functional).

{{function_node __wrapped__Cast_device_/job:localhost/replica:0/task:0/device:CPU:0}} Cast int32 to string is not supported [Op:Cast] name: 

Call arguments received by layer 'model' (type Functional):
  • inputs=tf.Tensor(shape=(1, 512), dtype=int32)
  • training=None
  • mask=None